Paul O'Leary

Comp 4448

July 19, 2021

MINI-Project FROM Assignment 3

In [1]:
# Import the usual suspects

import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns

Mini Project

In [2]:
# Survival from Malignant Melanoma, based on Age, Sex, and indicator of ulceration.

Survival from Malignant Melanoma
Description
The melanoma data frame has 205 rows and 7 columns.

The data consist of measurements made on patients with malignant melanoma. Each patient had their tumour removed by surgery at the Department of Plastic Surgery, University Hospital of Odense, Denmark during the period 1962 to 1977. The surgery consisted of complete removal of the tumour together with about 2.5cm of the surrounding skin. Among the measurements taken were the thickness of the tumour and whether it was ulcerated or not. These are thought to be important prognostic variables in that patients with a thick and/or ulcerated tumour have an increased chance of death from melanoma. Patients were followed until the end of 1977.

Usage
melanoma
Format
This data frame contains the following columns:

time
Survival time in days since the operation, possibly censored.

status
The patients status at the end of the study. 1 indicates that they had died from melanoma, 2 indicates that they were still alive and 3 indicates that they had died from causes unrelated to their melanoma.

sex
The patients sex; 1=male, 0=female.

age
Age in years at the time of the operation.

year
Year of operation.

thickness
Tumour thickness in mm.

ulcer
Indicator of ulceration; 1=present, 0=absent.

Source
The data were obtained from

Andersen, P.K., Borgan, O., Gill, R.D. and Keiding, N. (1993) Statistical Models Based on Counting Processes. Springer-Verlag.

In [4]:
data = pd.read_csv("melanoma.csv")

In [97]:
data.head(7)

,Unnamed: 0,time,status,sex,age,year,thickness,ulcer
0,1,10,3,1,76,1972,6.76,1
1,2,30,3,1,56,1968,0.65,0
2,3,35,2,1,41,1977,1.34,0
3,4,99,3,0,71,1968,2.90,0
4,5,185,1,1,52,1965,12.08,1
5,6,204,1,1,28,1971,4.84,1
6,7,210,1,1,77,1972,5.16,1


a.  Cleanup

In [72]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205 entries, 0 to 204
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  205 non-null    int64  
 1   time        205 non-null    int64  
 2   status      205 non-null    int64  
 3   sex         205 non-null    int64  
 4   age         205 non-null    int64  
 5   year        205 non-null    int64  
 6   thickness   205 non-null    float64
 7   ulcer       205 non-null    int64  
dtypes: float64(1), int64(7)
memory usage: 12.9 KB


In [99]:
# Not missing any data.

# Let's only keep the columns to be used

data = data[["sex", "age", "ulcer", "status"]]

In [100]:
data.head(5)

,sex,age,ulcer,status
0,1,76,1,3
1,1,56,0,3
2,1,41,0,2
3,0,71,0,3
4,1,52,1,1


For the purposes of this little study, I will change the statuses of 2 (Still alive) and 3 (Died of other causes) to be 2 (Did not die due to Melanoma).

In [102]:
data['status'] = data['status'].replace([3], 2)
data.head(5)

,sex,age,ulcer,status
0,1,76,1,2
1,1,56,0,2
2,1,41,0,2
3,0,71,0,2
4,1,52,1,1


In [76]:
# Scale the data - the ages have a much larger range than the other columns.
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
data_scaled = pd.DataFrame(scaler.fit_transform(data),columns = data.columns)

In [103]:
data_scaled.head(5)

,sex,age,ulcer,status
0,1.0,0.791209,1.0,1.0
1,1.0,0.571429,0.0,1.0
2,1.0,0.406593,0.0,1.0
3,0.0,0.736264,0.0,1.0
4,1.0,0.527473,1.0,0.0


Age was obviously scaled from 0 - 1, and status was simply changed to 0 and 1, instead of 1 and 2.  The other columns are unchanged.

c. Data Splitting

In [78]:
from sklearn.model_selection import train_test_split

feat = data_scaled.drop("status",axis=1)
targ = data_scaled["status"].copy()

X_train, X_test, y_train, y_test = train_test_split(feat, targ, test_size=0.3, random_state=42)

In [79]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(143, 3)
(143,)
(62, 3)
(62,)


d. Model Construction

In [80]:
# Construct the models

In [81]:
# Create a decision tree classifier

clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)


# Create a random forest classifier
from sklearn.ensemble import RandomForestClassifier

clf_rf = RandomForestClassifier()
clf_rf = clf_rf.fit(X_train, y_train)


Some Accuracy scores before any tuning

In [82]:
y_pred_dt = clf.predict(X_test)
y_pred_rf = clf_rf.predict(X_test)

# Save the pre-tuned results for later
res_dt_pre = accuracy_score(y_test, y_pred_dt)
res_rf_pre = accuracy_score(y_test, y_pred_rf)

print("Accuracy Score for the Test set for the Decision Tree Classifier: ", res_dt_pre)
print("Accuracy Score for the Test set for the Random Corest Classifier: ", res_rf_pre)

Accuracy Score for the Test set for the Decision Tree Classifier:  0.6290322580645161
Accuracy Score for the Test set for the Random Corest Classifier:  0.6129032258064516


e.  Hyperparameter tuning

In [83]:
# What are the hyperparams for both?

clf.get_params()

{'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'random_state': None,
 'splitter': 'best'}

In [84]:
clf_rf.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [85]:
clf.tree_.max_depth

11

In [86]:
# For Decision Tree, try a grid_search

# Provided
param_grid = {"max_depth":range(1, clf.tree_.max_depth), "max_features":[0.2,0.4,0.6,0.8]}

grid = GridSearchCV(estimator=clf, param_grid=param_grid, scoring = None, cv=8)
grid = grid.fit(X_train, y_train)

print("Optimal: ", grid.best_params_)

Optimal:  {'max_depth': 1, 'max_features': 0.4}


In [87]:
# Now for Random Forest
# Let's try n_estimators and max_depth

n_est = [5, 10, 25, 50]
max_depth = [2, 4, 8, 15]

param_grid = {"n_estimators":n_est, "max_depth":max_depth}

grid_rf = GridSearchCV(estimator=clf_rf, param_grid=param_grid, scoring = None, cv=8)
grid_rf = grid_rf.fit(X_train, y_train)

print("Optimal: ", grid_rf.best_params_)

Optimal:  {'max_depth': 2, 'n_estimators': 10}


f.  Build the new models, based on the tuning checks

In [88]:
# Create a decision tree classifier

clf_tuned = tree.DecisionTreeClassifier(max_depth=1, max_features=0.4)
clf_tuned = clf_tuned.fit(X_train, y_train)


# Create a random forest classifier
from sklearn.ensemble import RandomForestClassifier

clf_rf_tuned = RandomForestClassifier(max_depth=2, n_estimators=50)
clf_rf_tuned = clf_rf_tuned.fit(X_train, y_train)


In [89]:
y_pred_dt_tuned = clf_tuned.predict(X_test)
y_pred_rf_tuned = clf_rf_tuned.predict(X_test)

# Accuracy results after tuning
res_dt_post = accuracy_score(y_test, y_pred_dt_tuned)
res_rf_post = accuracy_score(y_test, y_pred_rf_tuned)


print("Accuracy Score for the Test set for the Decision Tree Classifier: ", res_dt_post)
print("Accuracy Score for the Test set for the Random Corest Classifier: ", res_rf_post)

Accuracy Score for the Test set for the Decision Tree Classifier:  0.7580645161290323
Accuracy Score for the Test set for the Random Corest Classifier:  0.7741935483870968


Interesting that both methods have exactly the same results...

Based on the chosen hyperparameters, here are the changes in the results:

In [90]:
print("Decision Tree Classifier Accuracy before tuning: ", res_dt_pre, " and after tuning: ", res_dt_post)
print("Random Forest Classifier Accuracy before tuning: ", res_rf_pre, " and after tuning: ", res_rf_post)

Decision Tree Classifier Accuracy before tuning:  0.6290322580645161  and after tuning:  0.7580645161290323
Random Forest Classifier Accuracy before tuning:  0.6129032258064516  and after tuning:  0.7741935483870968


g. Overfitting?

What do the models' performance look like on the training data?

In [92]:
y_pred_dt_tuned_trainset = clf_tuned.predict(X_train)
y_pred_rf_tuned_trainset = clf_rf_tuned.predict(X_train)

# Accuracy results after tuning
res_dt_post_trainset = accuracy_score(y_train, y_pred_dt_tuned_trainset)
res_rf_post_trainset = accuracy_score(y_train, y_pred_rf_tuned_trainset)

print("Accuracy, DT, Train Set: ", res_dt_post_trainset)
print("Accuracy, RF, Train Set: ", res_rf_post_trainset)

Accuracy, DT, Train Set:  0.7202797202797203
Accuracy, RF, Train Set:  0.7132867132867133


Oddly, these models perform slightly less well on the Training set than on the Test set.  There is no overfitting, since the Training set performance is not better than the Test set performance by a large amount.  

The tuned models predicted the correct outcome 75% of the time, which is obviously quite a bit better than a coin toss.  However, I believe that this could be improved, possibly with more hyperparameter tuning.  This dataset only contained 205 records - a larger set may result in improved accuracy.  This student's selection of predictors - sex, age and presence of an ulceration - may have limited the accuracy.  Other measures not included in this dataset, such as family history, overall health, and if this was the first tumor removed - may improve accuracy as well.  


